<a href="https://colab.research.google.com/github/Rocks-n-Code/PythonCourse/blob/master/Salt_Cavern_Workflows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

<sub><sub>*Given by Agapito Associates, LLC on September 8, 2025 for Lousiana Department of Natrual Resources.*</sub></sub>

<table>
  <tr>
    <td><img src="https://www.agapito.com/wp-content/uploads/2025/05/TextBoxLogoWhitespace.png" height="75"></td>
    <td><h1 style="margin:0;">Python & Salt Caverns</h1></td>
    <td><img src="https://www.dnr.louisiana.gov/images/DENR-Seal.png" height="90"/>
  </tr>
</table>


___





# Introduction



The target audience of this course is someone with basic python knowledge. We'll go through some common workflows when working with salt caverns that are made easier and scaleable by using python.

1) GIS
- <sub>Preview lat/long locations with *folium*</sub>
- <sub>Well spot creation with *geopandas*</sub>
2) Well Deviation Surveys
- <sub>Well deviation survey with *wellpathpy*</sub>
3) Sonar Surveys
- <sub>Pulling files with *requests*</sub>
- <sub>Parsing text from a `pdf` with *pdfminer*</sub>
- <sub>Parsing text into a table with *pandas*</sub>
- <sub>Sonar file processing with *SaltPy*</sub>
4) 3D Models
- <sub>3D Visualization with *PyVista*</sub>

<br>

Instructor: [Matthew W. Bauer](https://www.agapito.com/team/matthew-bauer/)

<table>
  <tr>
    <td><img src="https://www.rmag.org/clientuploads/directory/board/2024_Board/Bauer_-_President_elect.png" width="90"></td>
    <td>
      <strong>Matthew W. Bauer, PG</strong><br>
      20 years in quanitative geoscience specializing in applications of python for geology. Projects ranging from the application<br> of spatial data science in solution mined caverns and injection wells, the evaluation of energy project economics, to the<br> spatiotemporal monitoring of land records and commodity prices. <br><br>• Affiliate Faculty at Colorado School of Mines <br>• President of Rocky Mountain Association of Geologists<br>• Short Course instructor for RMAG, AAPG, and GSA<br>• PG in KS & LS
  </tr>
</table>


---

In [ ]:
# Install packages not already on the COLAB instance

!pip install wellpathpy
!pip install pdfminer
!apt-get install -qq xvfb
!pip install pyvista panel -q
!pip install pyvista[jupyter]

In [ ]:
## Import the packages we will use.

# General
import pandas as pd
import numpy as np

# GIS
import geopandas as gpd
from shapely import Point, LineString

# Web Mapping
import folium
import folium.plugins as plugins

# Well Deviation Surveys
import wellpathpy as wp

# Web & PDFs
import io
import requests
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

# Visualization
import pyvista as pv


# Set options
pd.set_option('display.max_columns', None)

# Needed for running in colab
pv.global_theme.jupyter_backend = 'static'
pv.global_theme.notebook = True
pv.start_xvfb()
pv.set_jupyter_backend('html')

___
#

<table>
  <tr>
    <td><img src="https://www.energy.gov/sites/default/files/styles/full_article_width/public/2025-03/Bayou%20Choctaw.png" width="100"></td>
    <td style="padding-left:10px;"><h2 style="margin:0;">Section 1: Make a Well Spot Shapefile</h2></td>
  </tr>
</table>

Let's make a GIS file for the location of well. In this case we will use PPG007B as an example.




*All data is publically avalible through [SONRIS](https://sonlite.dnr.state.la.us/ords/f?p=108:2).*

[PPG 007B Deviation Survey](https://sonlite.dnr.state.la.us/dnrservices/redirectUrl.jsp?dDocname=14325104&showInline=True)



In [ ]:
# X,Y, and Z from the deviation survey
x,y,z = 2623600.00, 643684.00, 12.6

# Making the GeoPandas object
crs = 'EPSG:3452'
point = Point(x,y,z)
print(point)

wells = gpd.GeoDataFrame({'Well_No':['PPG-007B'],
                          'serial_no':[67270],
                          'geometry':[point]},
                         crs=crs)

# Preview the GeoDataFrame
wells.plot()

In [ ]:
# Change Coordinate Reference System
wells_wgs84 = wells.to_crs('EPSG:4326')
point = wells_wgs84.at[0,'geometry']
x0,y0 = [p[0] for p in point.xy]
print(x0,y0)

## View with Folium
m = folium.Map(location=[y0,x0],
               tiles='Cartodb dark_matter',
               zoom_start=10)

# Add a marker
folium.Marker(
              [y0,x0],
              popup='PPG 007B',
              icon=plugins.BeautifyIcon(
                              icon="circle-small",
                              icon_shape="circle",
                              border_color='blue',
                              background_color='lightblue'
                          )
              ).add_to(m)

m

In [ ]:
## Save File
wells.to_file('ExampleWells.shp')
wells_wgs84.to_file('ExampleWells_WSG84.gpkg')

___
#
<table>
  <tr>
    <td><img src="https://www.energy.gov/sites/default/files/styles/full_article_width/public/2025-03/Big%20Hill.png" width="100"></td>
    <td style="padding-left:10px;"><h2 style="margin:0;">Section 2: Process a Well Deviation Survey</h2></td>
  </tr>
</table>


For well deviation surveys that are not in ascii text I really like the snip tool in Windows 11 that has OCR. I've pasted the data below for you.

In [ ]:
# Deviation Survey Raw Data
lines = '''0.0
100.0
200.1
300.0
400.0
500.0
600.0
700.0
800.0
900.
999.9
1100.0
1200.0
1300.0
1400.0
1500.0
1600.0
1700.0
1800.0
1899.9
2000.0
2100.0
2200.0
2300.0
2400.0
2501.0
0.0
267.3
265.3
268.3
253.3
264.3
255.3
354.3
343.3
330.3
282.3
277.3
275.3
284.3
294.3
345.3
345.3
354.3
25.3
34.3
27.3
42.3
50.3
42.3
48.3
104.3
0.0
0.3
0.2
0.2
0.3
0.4
0.2
0.4
0.8
0.7
1.9
1.6
1.5
1.1
1.0
0.9
0.8
0.6
0.4
0.5
0.4
0.5
0.4
0.3
0.2
0.8'''.split('\n')

# Number of lines
n = int(len(lines) / 3)

# Seperate Columns
md = lines[:n]
azi = lines[n:n*2]
inc = lines[n*2:]

# Make DataFrame
dev = pd.DataFrame({'md':md, 'azi': azi, 'inc':inc})
dev = dev.astype(float)

dev.head()

In [ ]:
## Make a wellpathpy object with the raw data
dev = wp.deviation(md = dev.md,
                   inc = dev.inc,
                   azi = dev.azi)

# LAS files are often on 0.5 ft steps so lets resample to 1/2' intervals.
# Let's use that step distance so we could merge a well log LAS.
depths = np.arange(0, dev.md.max() + 1, 0.5)

pos = dev.minimum_curvature().resample(depths=depths)
dev2 = pos.deviation()

In [ ]:
# Let's make a dataframe of that data
devsurv = pd.DataFrame({'md':dev2.md, 'azi':dev2.azi, 'inc':dev2.inc,
                        'tvd':pos.depth, 'n':pos.northing, 'e':pos.easting})

# Add x,y,z data of where we are refrenced in space.
# Note: Be careful if the survey is from KB then you'll need the APD
devsurv['x'] = x + devsurv['e']
devsurv['y'] = y + devsurv['n']
devsurv['z'] = z - devsurv['tvd']

# We want to know in 3D space how any MD has moved from straight down to
#  process our sonars. Let's add those columns
devsurv['dx'] = devsurv['e']
devsurv['dy'] = devsurv['n']
devsurv['dz'] = devsurv['md'] - devsurv['tvd']

# Geometry
devsurv['geometry'] = gpd.points_from_xy(devsurv.x, devsurv.y, devsurv.z)

# Change from pandas.DataFrame to geopandas.GeoDataFrame
devsurv = gpd.GeoDataFrame(devsurv, geometry='geometry', crs=wells.crs)

# Plot & Preview Points
ax = devsurv.plot(c=devsurv['z'], alpha=0.3, markersize=0.5)
wells.plot(color='white', edgecolor='k', ax=ax)

devsurv.tail()

In [ ]:
## Make Lines GIS File
line = LineString(devsurv.geometry)

# Preview line
line

In [ ]:
# Make Deviation Survey Line GIS
lines = wells.copy()
lines['geometry'] = line

# Plot and Preview Line
ax = lines.plot()
wells.plot(color='white', edgecolor='k', ax=ax)

In [ ]:
## Save out files
devsurv.to_file('ExampleDevPnts.gpkg')
lines.to_file('ExampleDevLine.gpkg')

___

#

<table>
  <tr>
    <td><img src="https://www.energy.gov/sites/default/files/styles/full_article_width/public/2025-03/Bryan%20Mound.png" width="100"></td>
    <td style="padding-left:10px;"><h2 style="margin:0;">Section 3: Processing Cavern Sonar Surveys</h2></td>
  </tr>
</table>

Sonar come in several flavors unique to the vedors who ran the survey. Sonarwire, now part of Empire Wireline is one of the most common vendors in the United States. Their `cwr` file can be converted with their software *Sonar Post Processing* to export an ascii file known as a `lwt` (long wall table). The `lwt` format is optimized for printing and has a table per depth and inclination with the ranges arranged in a base azimuth row and addition azimuth column. This `lwt` is found, with small variations, is found with other vendors. We move these

Agapito has been internally develping python package `SonarPy` which is planned for public release in 2026. We will outline the workflow of processing with `SaltPy`.

---



## Parsing a PDF with Python

We will start with an example that occurs all to often where we are not provided an original file from the vendor and we but convert a pdf into a usable format. [Public data source example: PPG 007B April 22, 2025](https://sonlite.dnr.state.la.us/dnrservices/redirectUrl.jsp?dDocname=15234027&showInline=True)

In [ ]:
def pull_pdf_lwt_SONIC_SURVEYS(url, start=0, end=9999):
    '''
    Parses SONIC SURVEY digital pdfs (does not work on scanned pdfs) for the text
    of the lwt pages. Optional page numbers speed this up.

    INPUTS
    url: url of document, redirects may not work
    start (optional): int, starting page index of lwt (first page is 0)
    end (optional): int, end page index of lwt ("Page 100" would be index 99)
    '''
    # Step 1: Download the PDF from the GitHub raw link
    response = requests.get(url)
    response.raise_for_status()  # Raise error if download fails

    # Step 2: Convert to a file-like object
    fp = io.BytesIO(response.content)

    # PDFMiner setup
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    # Extraction logic
    txtlst = []
    page_no = 0
    for pageNumber, page in enumerate(PDFPage.get_pages(fp)):
        if page_no < start:
            page_no += 1
            continue
        elif page_no > end:
            break
        elif pageNumber == page_no:
            interpreter.process_page(page)

            data = retstr.getvalue()
            t = data.encode('utf-8')
            if (b'DEPTH' in t) & (b'Bearing' in t) & (b'TILT' in t):
                txtlst.append(t)

            # Clear the StringIO buffer
            retstr.truncate(0)
            retstr.seek(0)

        page_no += 1

    return txtlst

rawurl = 'https://github.com/Rocks-n-Code/PythonCourse/blob/master/data/PPG7B_20250311_15234027~1.pdf?raw=true'
txtlst = pull_pdf_lwt_SONIC_SURVEYS(rawurl, start=61, end=128)

print('Pages',len(txtlst))

# Preview 1st page's text
txtlst[0]

---

## Parse Text with Python

PDF's often do not preserve data relationships of tables all that well. In this case we find that the "columns" run through the three tables on a page. This behavior is consistent so we can parse this with Python and make a transformed table that has one shot per row.

In [ ]:
def parse_txtlst_to_dfT(txtlst):
    '''
    Double new line character delimination for lines, columns extend through all
    tables

    '''

    # Make empty dataframe
    dfT = pd.DataFrame()

    # Iterrate through pages
    for tl in txtlst:

        # Convert bytes to string
        tl = tl.decode('utf-8')

        # Clean off header
        tl = tl[tl.index('DEPTH'):]

        # number of tables on the page
        ntbls = tl.count('DEPTH')

        ## Data in this example are in columns of all the tables.
        # Split Accordingly.
        data = [p.split('\n') for p in tl.split('\n\n')]

        # pad columns
        colmax = max([len(p) for p in data])
        data = [[np.nan]+p if len(p) < colmax else p for p in data ]

        # drop last (short) one
        data = [x for x in data if len(x) == colmax]

        # Make array for easier parsing
        a = np.array(data)

        # Iterrate through the three tables
        for i in range(0, ntbls):
            #print(i)
            a0 = a[i::3]
            headline = a0[:,0]
            headline = [x for x in headline if x != 'nan']
            header = {str(k):str(v) for k,v in zip(headline[::2], headline[1::2])}

            rs = a0[1:,2:].astype(float).T.flatten()
            n = rs.shape[0]
            step = 360 / n
            azi = np.arange(0,360,step)

            # Make dataframe of each table
            temp = pd.DataFrame({'depth':[header['DEPTH']]*n,
                                'tilt':[header['TILT']]*n,
                                'vos':[header['VOS']]*n,
                                'cAzi':azi,
                                'r':rs})

            # Add table transformed to the end of dfT
            dfT = pd.concat([dfT,temp], ignore_index=True)

    dfT = dfT.astype(float)

    return dfT

# Use function to make dfT
dfT=parse_txtlst_to_dfT(txtlst)

# Preview transformed lwt dataframe (one shot per row)
dfT

In [ ]:
# Save out for use with SaltPy
dfT.to_csv('PPG7B_20250311_dfT.csv', index=False)

---

## SaltPy

While we're close to release but not quite there. We'll move to presenting from a jupyter notebook for this section.

<img src="https://jupyter.org/assets/homepage/main-logo.svg" alt="jupyter" width="300"/>


---

#

<table>
  <tr>
    <td><img src="https://www.energy.gov/sites/default/files/styles/full_article_width/public/2025-03/West%20Hackberyy.png" width="100"></td>
    <td style="padding-left:10px;"><h2 style="margin:0;">Section 4: 3D Modeling with PyVista</h2></td>
  </tr>
</table>


Moving our visual representations beyond "paper space" in two dimentions is a powerful tool to help understand anisotropic geologic systems and our interactions with them.  There are several packages out there that will let you make models in three dimensions but my current favorite is `pyvista`. [PyVista](https://pyvista.org/) package is built on top of VTK and allows for a wide amount of flexibility with working on 3D data, visualization, and sharing models with others.  We will make a basic model and chat about the components but I encourage you to look through the documentation and see how you can build a beautiful tool to communicate with others what is going on in your own project.

In [ ]:
# Download the exported stl file
url = "https://raw.githubusercontent.com/Rocks-n-Code/PythonCourse/453ab373e163f167bca7e11b184d9fcc7c5cd70d/data/PPG007B_nosmoothing_meters.stl"
filename = "PPG007B_nosmoothing_meters.stl"

response = requests.get(url)
with open(filename, "wb") as f:
    f.write(response.content)

# Open stl with pyvista
mesh = pv.read(filename)

In [ ]:
# Plot only one element
mesh.plot()

In [ ]:
# Convert other inputs into the same CRS
crs = 'EPSG:32615'

# Geopandas can handle x,y coordinate refrance system changes but does not
# change the z axis

# well deviation line
linesUTM = lines.to_crs(crs)
devcoords = np.array(linesUTM.at[0,'geometry'].coords[:])
devcoords[:,2] *= 0.3048

# well surface location
wellsUTM = wells.to_crs(crs)
x0,y0,z0 = wellsUTM.at[0,'geometry'].coords[:][0]
z0 *= 0.3048

In [ ]:
# Functions for making well symbol

def make_circ(xyz=(0,0,0), r=5):
    '''
    Makes coordinates of 16 sided "circle"

    Inputs:
    xyz, tuple, (X, Y, Z) of center of circle
    r, (optional) float, radius of circle

    Output:
    coords, list of x,y,z coordinates
    '''
    x0,y0,z0 = xyz
    num_segments = 16

    # Generate angles for each segment
    angles = np.linspace(0, 2*np.pi, num_segments, endpoint=False)

    # Compute (x, y) coordinates for each angle
    x_coordinates = r * np.cos(angles)
    y_coordinates = r * np.sin(angles)

    # Combine x and y coordinates
    coordinates = np.column_stack((x_coordinates, y_coordinates))
    coords =  np.round(coordinates,5) + np.array([x0,y0])
    coords = [(p[0],p[1],z0) for p in coords]
    coords = coords + coords[0:1]
    return coords

def make_cross(xyz=(0,0,0),r=10):
    '''
    Makes coordinates of cross

    Inputs:
    xyz, tuple, (X, Y, Z) of center of cross
    r, (optional) float, radius of cross

    Output:
    list of x,y,z coordinates
    '''
    x0,y0,z0 = xyz
    return [[(x0 + r,y0,z0),(x0 - r,y0,z0)],[(x0, y0 + r,z0),(x0,y0 - r,z0)]]

def make_well_mark(xyz=(0,0,0),r=10):
    '''
    Makes coordinates of well symbol

    Inputs:
    xyz, tuple, (X, Y, Z) of center of cross
    r, (optional) float, radius of cross

    Output:
    cross
    '''
    cross = make_cross(xyz=xyz,r=r)
    cross.append(make_circ(xyz=xyz,r=r/2))
    return cross

In [ ]:
# Make Plotter Plot of Multiple Elements
pl = pv.Plotter()

# Add well point
linescoords = make_well_mark(xyz=(x0,y0,z0))

for line in linescoords:
    line_source = pv.MultipleLines( points=line)
    pl.add_mesh(line_source, color="k", line_width=1)

# Add Well Deviation path
devsource = pv.MultipleLines( points=devcoords)
pl.add_mesh(devsource, color="grey", line_width=3, opacity=0.3)

# Add Cavern
pl.add_mesh(mesh, color='r', opacity=0.3)

pl.show()

In [ ]:
# Save an html file of our 3D model
pl.camera_position ='xz' #adjust camera location
pl.export_html(r"PPG-007B_example.html")

You can access the file in the folder in the left bar and download it.
This file can be shared with others to view 3d models without special
software.